In [1]:
import os

In [2]:
%pwd

'/Users/macbookpro/Desktop/MLProjects/Text-Summarizer-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/macbookpro/Desktop/MLProjects/Text-Summarizer-Project'

In [11]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [14]:
class ModelEvaluation:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        create_directories([self.config.model_evaluation.root_dir])

    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="dialogue", 
                               column_summary="summary"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate_(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_evaluation.tokenizer_path)
        model_distilbart = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_evaluation.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.model_evaluation.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_distilbart, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        #rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index = ['distilbart'] )
        df.to_csv(self.config.model_evaluation.metric_file_name, index=False)

        



In [15]:
try:
    model_evaluation_config = ModelEvaluation()
    model_evaluation_config.evaluate_()
except Exception as e:
    raise e

[2025-09-05 11:35:13,233: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-05 11:35:13,244: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-05 11:35:13,254: INFO: common: created directory at: artifacts]
[2025-09-05 11:35:13,257: INFO: common: created directory at: artifacts/model_evaluation]


/Applications/anaconda3/envs/textSumm/lib/python3.8/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
100%|██████████| 5/5 [12:52<00:00, 154.55s/it]


[2025-09-05 11:48:09,749: INFO: rouge_scorer: Using default tokenizer.]
